In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

In [ ]:
import os
from pathlib import Path


fpath = (
    Path(os.environ["DSS_HOME"])
    / "toxicainment/2025-02-07-saxony-labeled-data/human-labels.csv"
)
human_labels = pd.read_csv(fpath, dtype={"post_id": str})
human_labels = (
    human_labels.sort_values(by="timestamp")
    .rename(
        columns={
            "is_saxony_election": "is_saxony",
            "is_saxony_election_comment": "is_saxony_comment",
        }
    )  # The column name was wrong while saving, but the UI correctly displayed that it does not HAVE to be politics in saxony
    .groupby(["post_id", "classification_by"])
    .last()
    .reset_index()
)
print(len(human_labels))
# join num raters per post
nraters_by_post = (
    human_labels.groupby("post_id", as_index=False)["classification_by"]
    .agg(["nunique"])
    .rename(columns={"nunique": "nraters"})
)
human_labels = pd.merge(human_labels, nraters_by_post, on="post_id", how="left")
human_labels.head()

In [4]:
questions = [
    "is_political",
    "is_saxony",
    "is_intolerant",
    "is_hedonic_entertainment",
    "is_eudaimonic_entertainment",
]
comment_cols = [f"{col}_comment" for col in questions]


# Dataset Balance

In [ ]:
label_balance = (
    human_labels[questions].apply(lambda s: s.value_counts(normalize=True)).round(3)
)
label_balance


In [ ]:
from bench_lib.evaluation import plot_scalars_for_questions

plot_scalars_for_questions(label_balance.max(axis=0), questions, "Majority Class %")

# Percentage Agreement


In [ ]:
def compute_agreement(human_labels: pd.DataFrame, question: str) -> float:
    df = pd.crosstab(human_labels["post_id"], human_labels[question])
    full_agreement = (df["no"] == 0) | (df["yes"] == 0)
    return float(full_agreement.mean())


agreements = [compute_agreement(human_labels, q) for q in questions]
print(agreements)
plot_scalars_for_questions(agreements, questions, "Percentage Agreement")


# Krippendorff's alpha

In [ ]:
import matplotlib.pyplot as plt

from bench_lib.evaluation import krippendorf_alpha


alphas = [
    krippendorf_alpha(human_labels["post_id"], human_labels[q]) for q in questions
]
plot_scalars_for_questions(alphas, questions, "Krippendorff's alpha")

In [ ]:
[round(a, 2) for a in alphas]

# Fleis Kappa

In [ ]:
from bench_lib.evaluation import fleiss_kappa

n_raters = 3
q = "is_intolerant"
fk_human_labels = human_labels.query("nraters == @n_raters")
table = pd.crosstab(fk_human_labels["post_id"], fk_human_labels[q]).values
kp, agreements = fleiss_kappa(table, method="fleiss")
ag_counts = pd.Series(agreements).value_counts()
print(kp)
fig = plt.figure(figsize=(4, 3))
plt.bar(ag_counts.index, ag_counts.values, width=0.05)
plt.xlim(-0.1, 1.1)
plt.xlabel("Agreement")
plt.ylabel("Count")
plt.title(f"Agreement for '{q}' with {n_raters} raters. Fleiss Kappa: {kp:.2f}")
plt.tight_layout()
plt.show()

# Post Difficulty

In [ ]:
from matplotlib import pyplot as plt
from bench_lib.evaluation import difficulty_score, get_means

q_mens = {q: get_means(human_labels, question=q) for q in questions}
difficulty = difficulty_score(pd.DataFrame(q_mens))

diff_counts = difficulty.value_counts()
fig = plt.figure()
plt.bar(diff_counts.index, diff_counts.values, width=0.03)
plt.xlabel("Difficulty")
plt.ylabel("Count")
plt.title("Post Difficulty")
plt.tight_layout()

In [ ]:
post_difficulty_granular = pd.melt(
    (human_labels.groupby("post_id")[questions].nunique() == 1).reset_index(),
    id_vars="post_id",
    value_vars=questions,
    value_name="human_consistent",
)
post_difficulty_granular

In [ ]:
unique_answers_by_post = (
    human_labels.groupby("post_id")[questions].nunique().max(axis=1)
)
hard_posts = unique_answers_by_post[unique_answers_by_post > 1]
easy_posts = unique_answers_by_post[unique_answers_by_post == 1]
print(len(hard_posts), len(easy_posts))

In [ ]:
post_difficulty = pd.concat(
    [
        pd.DataFrame({"post_id": hard_posts.index, "difficulty": "hard"}),
        pd.DataFrame({"post_id": easy_posts.index, "difficulty": "easy"}),
    ]
)
post_difficulty

In [ ]:
post_difficulty["difficulty"].value_counts()

# How does AI perform on the easy posts?
We assume here the ground truth to be the human labels, which are unique by construction

In [ ]:
from bench_lib.evaluation import load_ai_labels, compute_ai_perfs


human_labels_easy_set = human_labels.query("post_id in @easy_posts.index")[
    ["post_id", *questions]
].drop_duplicates()
human_labels_easy_set

gemma3_folders = [
    "gemma-3-4b-it",
    "gemma-3-12b-it",
    "gemma-3-27b-it_00",
    "gemini-2.0-flash-001",
]
gemma3_ai_labels = load_ai_labels(gemma3_folders, questions, comment_cols)
gemma3_ai_perfs = compute_ai_perfs(human_labels_easy_set, gemma3_ai_labels, questions)

In [ ]:
from bench_lib.evaluation import plot_ai_perfs


g3_order = [f"google/gemma-3-{n}b-it" for n in [4, 12, 27]] + [
    "google/gemini-2.0-flash-001"
]
plot_ai_perfs(gemma3_ai_perfs, g3_order, questions)

In [ ]:
qwen_ai_labels = load_ai_labels(
    folders=["qwen-2.5-vl", "gemini-2.0-flash-001"],
    questions=questions,
    comment_cols=comment_cols,
)
qwen_ai_labels.query(
    "`Model ID`.str.contains('Qwen') or `Model ID`.str.contains('gemini')", inplace=True
)
qwen_ai_perfs = compute_ai_perfs(human_labels_easy_set, qwen_ai_labels, questions)
order = [f"Qwen/Qwen2.5-VL-{n}B-Instruct" for n in [3, 7, 7]] + ["gemini-2.0-flash-001"]
plot_ai_perfs(qwen_ai_perfs, order, questions)

# How does AI perform on the hard posts?
The hard posts cannot be evaluated on single-labels ground truth like the easy posts.
Instead we compare how often AI changes its mind, i.e. how self-consistent its answers are.

In [ ]:
folders = [f"gemma-3-27b-it_{i:02d}" for i in range(3)]
ai_labels = load_ai_labels(folders, questions, comment_cols)

In [ ]:
ai_answers_hard = (
    pd.melt(ai_labels, id_vars="post_id", value_vars=questions)
    .groupby(["post_id", "variable"], as_index=False)["value"]
    .agg([("n_answers", "count"), ("n_unique", "nunique")])
    .assign(ai_consistent=lambda df: df["n_unique"] == 1)
)

ai_answers_hard = pd.merge(ai_answers_hard, post_difficulty_granular, on=["post_id", "variable"])
ai_answers_hard
# ai_answers_hard = pd.merge(ai_answers_hard, post_difficulty, on="post_id", how="left")
# ai_answers_hard.groupby(["variable", "is_easy"])["self_consistent"].value_counts(
#     normalize=True
# )

In [ ]:
ai_answers_hard.groupby(["variable", "human_consistent"])["ai_consistent"].value_counts(normalize=True)

# Regression
* Detecting Hedonic entertainment and Intolerance significantly decrease the self-consistency of AI
* Against my expectations, there is no significant effect of human consistency on AI consistency (hypothesis was human are inconsistent in difficutl posts, and so will be AI)


In [ ]:
# Perform logistic regression to analyze how variable type and human consistency
# affect AI consistency
import statsmodels.api as sm
import statsmodels.formula.api as smf
import numpy as np

# Convert categorical variable to dummy variables
model_data = ai_answers_hard.assign(
    variable=lambda df: pd.Categorical(df["variable"]),
    human_consistent=lambda df: df["human_consistent"].astype(int),
    ai_consistent=lambda df: df["ai_consistent"].astype(int),
)

# Fit logistic regression model
logit_model = smf.logit("ai_consistent ~ C(variable) + C(human_consistent)", data=model_data)
result = logit_model.fit()
print("Logistic Regression Results:")
print(result.summary())

# Display odds ratios
print("\nOdds Ratios:")
print(np.exp(result.params))

# When is AI inconsistent?

In [ ]:
idx = 2
row = ai_answers_hard.query("~ai_consistent").iloc[idx]
ai_labels.query("post_id == @row.post_id")[[row["variable"], f"{row['variable']}_comment"]].to_dict(orient="records")